In [1]:
from pyspark.sql import SparkSession
from delta import *

In [2]:
builder = SparkSession.builder.appName("MetricsOutputDemo") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")\
    .config("spark.sql.debug.maxToStringFields", "1000")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/Users/wgirten/anaconda/envs/jupyterlab/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/wgirten/.ivy2/cache
The jars for the packages stored in: /Users/wgirten/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-bae32a0e-9951-43ff-befa-b320d33f1294;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.1.0 in central
	found io.delta#delta-storage;2.1.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
:: resolution report :: resolve 143ms :: artifacts dl 6ms
	:: modules in use:
	io.delta#delta-core_2.12;2.1.0 from central in [default]
	io.delta#delta-storage;2.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	org.codehaus.jackson#jackson-core-asl;1.9.13 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded

22/09/28 12:09:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
DeltaTable.createIfNotExists(spark) \
    .tableName("default.airbnb_nyc") \
    .addColumn("listing_id", "INT") \
    .addColumn("description", "STRING") \
    .addColumn("neighbourhood_group", "STRING") \
    .addColumn("neighbourhood", "STRING") \
    .addColumn("room_type", "STRING") \
    .addColumn("price", "INT") \
    .execute()

In [4]:
# NYC Airbnb open dataset:
# https://www.kaggle.com/datasets/dgomonov/new-york-city-airbnb-open-data?select=AB_NYC_2019.csv
listings_data_1 = [
    (2539,'Clean & quiet apt home by the park','Brooklyn','Kensington','Private room',149),
    (5121,'BlissArtsSpace!','Brooklyn','Bedford-Stuyvesant','Private room',60),
    (5178,'Large Furnished Room Near Bway' ,'Manhattan','Hells Kitchen','Private room',79),
    (5203,'Cozy Clean Guest Room - Family Apt','Manhattan','Upper West Side','Private room',79),
    (5238,'Cute & Cozy Lower East Side 1 bdrm','Manhattan','Chinatown','Entire home/apt',150)
]
listings_data_2 = [
    (2539,'Price Reduced! Clean & quiet apt home by the park','Brooklyn','Kensington','Private room',135),
    (2595,'Skylit Midtown Castle','Manhattan','Midtown','Entire home/apt',225),
    (3647,'THE VILLAGE OF HARLEM....NEW YORK !','Manhattan','Harlem','Private room',150),
    (3831,'Cozy Entire Floor of Brownstone','Brooklyn','Clinton Hill','Entire home/apt',89),
    (5022,'Entire Apt: Spacious Studio/Loft by central park','Manhattan','East Harlem','Entire home/apt',80)
]
schema = 'listing_id int, description string, neighbourhood_group string, ' \
         'neighbourhood string, room_type string, price int'

In [5]:
listings_df = spark.createDataFrame(listings_data_1, schema)
listings_df.write.format("delta").mode("append").saveAsTable("default.airbnb_nyc")
listings_df.show()

+----------+--------------------+-------------------+------------------+---------------+-----+
|listing_id|         description|neighbourhood_group|     neighbourhood|      room_type|price|
+----------+--------------------+-------------------+------------------+---------------+-----+
|      2539|Clean & quiet apt...|           Brooklyn|        Kensington|   Private room|  149|
|      5121|     BlissArtsSpace!|           Brooklyn|Bedford-Stuyvesant|   Private room|   60|
|      5178|Large Furnished R...|          Manhattan|     Hells Kitchen|   Private room|   79|
|      5203|Cozy Clean Guest ...|          Manhattan|   Upper West Side|   Private room|   79|
|      5238|Cute & Cozy Lower...|          Manhattan|         Chinatown|Entire home/apt|  150|
+----------+--------------------+-------------------+------------------+---------------+-----+



In [6]:
spark.sql("""
   UPDATE airbnb_nyc
      SET description='Blissful Art Space!'
    WHERE listing_id=5121
""").show()

+-----------------+
|num_affected_rows|
+-----------------+
|                1|
+-----------------+



In [7]:
spark.sql(f"""
  DELETE FROM airbnb_nyc
   WHERE neighbourhood='Hells Kitchen'
  """).show()

+-----------------+
|num_affected_rows|
+-----------------+
|                1|
+-----------------+



In [8]:
spark.createDataFrame(listings_data_2, schema).createOrReplaceTempView("change_set")
spark.sql("""
   MERGE INTO airbnb_nyc AS target
      USING change_set AS source
      ON source.listing_id = target.listing_id
      WHEN MATCHED THEN UPDATE SET *
      WHEN NOT MATCHED THEN INSERT *
""").show()

+-----------------+----------------+----------------+-----------------+
|num_affected_rows|num_updated_rows|num_deleted_rows|num_inserted_rows|
+-----------------+----------------+----------------+-----------------+
|                5|               1|               0|                4|
+-----------------+----------------+----------------+-----------------+

